# MLI BYOR: Custom Explainers
This notebook is a demo of MLI **bring your own explainer recipe** (BYOR) Python API.

**Ad-hoc OOTB and/or custom explainer run** scenario:
* **Upload** interpretation recipe.
    * Determine recipe upload job **status**.
* **Run** ad-hoc recipe run job.
    * Determine ad-hoc recipe job **status**.
* **Get** explainer type for given job.
* **List** explanation types for given explainer run.
* **List** explanation representations (formats) URLs for given explanation type.
* **Download** explanation representation from ^ URL.    
* **Download** interpretation recipe job result **data**.

**Interpretation explainers run** scenario:
* **List** available/compatible explainers.
* **Choose** subset or all ^ compatible explainers.
* **Run** interpretation job.
   * Determine **status** of interpretation job.
   * Determine **status** per explainer job (running within the scope of interpretation).
* **List** explainer types which were ran within interpretation.
* **List** explainer runs for given explainer type within the interpretation.
* **List** explanation types for given explainer run.
* **List** explanation representations (formats) URLs for given explanation type.
* **Download** explanation representation from ^ URL.

### Virtual Environment and Dependencies
Prepare and activate virtual environment for this notebook by running:
```
. .env/bin/activate
pip install ipykernel
ipython kernel install --user --name=dai
```

In [1]:
import os
import pprint
import time
from random import randint
from h2oaicore.mli.oss.commons import MimeType, MliJobStatus
from h2oaicore.messages import (
    CommonDaiExplainerParameters,
    CommonExplainerParameters,
    DatasetReference,
    Explainer,
    ExplainerDescriptor,
    ExplainerJobStatus,
    ExplainerRunJob,
    ExplainersRunJob,
    InterpretationJob,
    ModelReference,
)

import h2o
import h2oai_client
from h2oai_client import Client

**Connect** to DAI server:

In [2]:
# connect to Driverless AI server - make sure to use the same 
# user name and password as when signing in through the GUI
hostname = '127.0.0.1'
address = 'http://' + hostname + ':12345'
username = 'h2oai'
password = 'h2oai'

# h2oai = Client("http://localhost:12345", "h2oai", "h2oai")
h2oai = Client(address = address, username = username, password = password)

# Upload 
Upload BYOR interpretation [recipe](http://192.168.59.141/mli-byor/mli_byor_foo.py) to Driverless AI server.

In [3]:
# URL of the recipe to upload

# Custom Morris sensitivity analysis
URL_BYOR_EXPLAINER = "https://h2o-public-test-data.s3.amazonaws.com/recipes/explainers/morris_sensitivity_explainer.py"
BYOR_EXPLAINER_NAME = "Morris Sensitivity Analysis"

**Upload recipe** to DAI server:

In [4]:
recipe_job_key = h2oai.create_custom_recipe_from_url(URL_BYOR_EXPLAINER)
recipe_job_key

'3782c148-017b-11eb-a66d-207918bc8e4b'

In [5]:
recipe_job = h2oai._wait_for_recipe_load(recipe_job_key)
pprint.pprint(recipe_job.dump())

{'created': 1601291428.8856423,
 'entity': {'data_file': '',
            'data_files': [],
            'datas': [],
            'explainers': [],
            'fpath': None,
            'key': '3782c148-017b-11eb-a66d-207918bc8e4b',
            'models': [],
            'name': 'morris_sensitivity_explainer',
            'pretransformers': [],
            'scorers': [],
            'transformers': [],
            'type': '',
            'url': 'https://h2o-public-test-data.s3.amazonaws.com/recipes/explainers/morris_sensitivity_explainer.py'},
 'error': '',
 'message': 'MorrisSensitivityLeExplainer:\nInstalling Class Packages',
 'progress': 1,
 'status': 0}


In [6]:
if recipe_job.entity.explainers:
    uploaded_explainer_id:str = recipe_job.entity.explainers[0].id    
else:
    # explainer already deployed (look it up)
    explainers = h2oai.list_explainers(
        experiment_types=None, 
        explanation_scopes=None,
        dai_model_key=None,
        keywords=None,
        explainer_filter=[]
    )
    uploaded_explainer_id = [explainer.id for explainer in explainers if BYOR_EXPLAINER_NAME == explainer.name][0]

print(f"Uploaded recipe ID: {uploaded_explainer_id}'")

Uploaded recipe ID: False_morris_sensitivity_explainer_d935205d_contentexplainer.MorrisSensitivityLeExplainer'


Driverless AI **model** and **dataset**:

In [7]:
# *) hardcoded DAI keys
# DATASET_KEY = "f12f69b4-475b-11ea-bf67-9cb6d06b189b"
# MODEL_KEY = "f268e364-475b-11ea-bf67-9cb6d06b189b"

# *) lookup compatible DAI keys
compatible_models = h2oai.list_explainable_models(
    explainer_id=uploaded_explainer_id, offset=0, size=30
)
if compatible_models.models:
    MODEL_KEY = compatible_models.models[0].key
    DATASET_KEY = compatible_models.models[0].parameters.dataset.key
else:
    raise RuntimeError("No compatible models found: please train an IID regression/binomial experiment")

target_col = h2oai.get_model_summary(key=MODEL_KEY).parameters.target_col
    
print(f"Model  : {MODEL_KEY}\nDataset: {DATASET_KEY}\nTarget : {target_col}")

Model  : 34edc42e-0152-11eb-a66d-207918bc8e4b
Dataset: dataset_33ab9348-0152-11eb-a66d-207918bc8e4b
Target : default payment next month


# List Explainers

List custom and OOTB recipes.

In [8]:
# list available server recipes
explainers = h2oai.list_explainers(
    experiment_types=None, 
    explanation_scopes=None,
    dai_model_key=None,
    keywords=None,
    explainer_filter=[]
)

for e in explainers:
    pprint.pprint(e.dump())

{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-sensitivity-analysis',
                   'formats': [],
                   'has_local': None,
                   'name': 'SaExplanation',
                   'scope': 'global'},
                  {'category': None,
                   'explanation_type': 'global-work-dir-archive',
                   'formats': [],
                   'has_local': None,
                   'name': 'WorkDirArchiveExplanation',
                   'scope': 'global'}],
 'id': 'h2oaicore.mli.byor.recipes.sa_explainer.SaExplainer',
 'keywords': ['run-by-default'],
 'model_types': ['iid'],
 'name': 'SA explainer'}
{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-disparate-impact-analysis',
                   'formats': [],
      

 'name': 'Relative permutation-based feature importance explainer'}
{'can_explain': ['regression', 'binomial', 'multinomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-feature-importance',
                   'formats': [],
                   'has_local': None,
                   'name': 'GlobalFeatImpExplanation',
                   'scope': 'global'},
                  {'category': None,
                   'explanation_type': 'local-feature-importance',
                   'formats': [],
                   'has_local': None,
                   'name': 'LocalFeatImpExplanation',
                   'scope': 'local'}],
 'id': 'h2oaicore.mli.byor.recipes.orig_kernel_shap_explainer.OriginalKernelShapExplainer',
 'keywords': [],
 'model_types': ['iid'],
 'name': 'Original Kernel SHAP explainer'}
{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope', 'local_scope'],
 'explanations'

                   'formats': [],
                   'has_local': None,
                   'name': 'ProxyExplanation',
                   'scope': 'local'}],
 'id': 'h2oaicore.mli.byor.recipes.transformed_shapley_explainer.TransformedShapleyFeatureImportanceExplainer',
 'keywords': ['run-by-default', 'proxy-explainer'],
 'model_types': ['iid'],
 'name': 'Transformed Shapley-based feature importance explainer'}
{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-feature-importance',
                   'formats': [],
                   'has_local': None,
                   'name': 'GlobalFeatImpExplanation',
                   'scope': 'global'}],
 'id': 'False_morris_sensitivity_explainer_d935205d_contentexplainer.MorrisSensitivityLeExplainer',
 'keywords': [],
 'model_types': ['iid'],
 'name': 'Morris Sensitivity Analysis'}
{'can_explain': ['regression', 'binomial', 'mul

In [9]:
# list server recipes for given experiment type
explainers = h2oai.list_explainers(
    experiment_types=["binomial"], 
    explanation_scopes=None,
    dai_model_key=None,
    keywords=None,
    explainer_filter=[]
)

for e in explainers:
    pprint.pprint(e.dump())

{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-sensitivity-analysis',
                   'formats': [],
                   'has_local': None,
                   'name': 'SaExplanation',
                   'scope': 'global'},
                  {'category': None,
                   'explanation_type': 'global-work-dir-archive',
                   'formats': [],
                   'has_local': None,
                   'name': 'WorkDirArchiveExplanation',
                   'scope': 'global'}],
 'id': 'h2oaicore.mli.byor.recipes.sa_explainer.SaExplainer',
 'keywords': ['run-by-default'],
 'model_types': ['iid'],
 'name': 'SA explainer'}
{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-disparate-impact-analysis',
                   'formats': [],
      

 'name': 'Relative permutation-based feature importance explainer'}
{'can_explain': ['regression', 'binomial', 'multinomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-feature-importance',
                   'formats': [],
                   'has_local': None,
                   'name': 'GlobalFeatImpExplanation',
                   'scope': 'global'},
                  {'category': None,
                   'explanation_type': 'local-feature-importance',
                   'formats': [],
                   'has_local': None,
                   'name': 'LocalFeatImpExplanation',
                   'scope': 'local'}],
 'id': 'h2oaicore.mli.byor.recipes.orig_kernel_shap_explainer.OriginalKernelShapExplainer',
 'keywords': [],
 'model_types': ['iid'],
 'name': 'Original Kernel SHAP explainer'}
{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope', 'local_scope'],
 'explanations'

                   'formats': [],
                   'has_local': None,
                   'name': 'ProxyExplanation',
                   'scope': 'local'}],
 'id': 'h2oaicore.mli.byor.recipes.transformed_shapley_explainer.TransformedShapleyFeatureImportanceExplainer',
 'keywords': ['run-by-default', 'proxy-explainer'],
 'model_types': ['iid'],
 'name': 'Transformed Shapley-based feature importance explainer'}
{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-feature-importance',
                   'formats': [],
                   'has_local': None,
                   'name': 'GlobalFeatImpExplanation',
                   'scope': 'global'}],
 'id': 'False_morris_sensitivity_explainer_d935205d_contentexplainer.MorrisSensitivityLeExplainer',
 'keywords': [],
 'model_types': ['iid'],
 'name': 'Morris Sensitivity Analysis'}
{'can_explain': ['regression', 'binomial', 'mul

In [10]:
# list server recipes compatible with given DAI model
explainers = h2oai.list_explainers(
    dai_model_key=MODEL_KEY, 
    experiment_types=None,
    explanation_scopes=None,
    keywords=None,
    explainer_filter=[]
)

for e in explainers:
    pprint.pprint(e.dump())

{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-sensitivity-analysis',
                   'formats': [],
                   'has_local': None,
                   'name': 'SaExplanation',
                   'scope': 'global'},
                  {'category': None,
                   'explanation_type': 'global-work-dir-archive',
                   'formats': [],
                   'has_local': None,
                   'name': 'WorkDirArchiveExplanation',
                   'scope': 'global'}],
 'id': 'h2oaicore.mli.byor.recipes.sa_explainer.SaExplainer',
 'keywords': ['run-by-default'],
 'model_types': ['iid'],
 'name': 'SA explainer'}
{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-disparate-impact-analysis',
                   'formats': [],
      

 'name': 'Relative permutation-based feature importance explainer'}
{'can_explain': ['regression', 'binomial', 'multinomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-feature-importance',
                   'formats': [],
                   'has_local': None,
                   'name': 'GlobalFeatImpExplanation',
                   'scope': 'global'},
                  {'category': None,
                   'explanation_type': 'local-feature-importance',
                   'formats': [],
                   'has_local': None,
                   'name': 'LocalFeatImpExplanation',
                   'scope': 'local'}],
 'id': 'h2oaicore.mli.byor.recipes.orig_kernel_shap_explainer.OriginalKernelShapExplainer',
 'keywords': [],
 'model_types': ['iid'],
 'name': 'Original Kernel SHAP explainer'}
{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope', 'local_scope'],
 'explanations'

                   'formats': [],
                   'has_local': None,
                   'name': 'ProxyExplanation',
                   'scope': 'local'}],
 'id': 'h2oaicore.mli.byor.recipes.transformed_shapley_explainer.TransformedShapleyFeatureImportanceExplainer',
 'keywords': ['run-by-default', 'proxy-explainer'],
 'model_types': ['iid'],
 'name': 'Transformed Shapley-based feature importance explainer'}
{'can_explain': ['regression', 'binomial'],
 'explanation_scopes': ['global_scope'],
 'explanations': [{'category': None,
                   'explanation_type': 'global-feature-importance',
                   'formats': [],
                   'has_local': None,
                   'name': 'GlobalFeatImpExplanation',
                   'scope': 'global'}],
 'id': 'False_morris_sensitivity_explainer_d935205d_contentexplainer.MorrisSensitivityLeExplainer',
 'keywords': [],
 'model_types': ['iid'],
 'name': 'Morris Sensitivity Analysis'}
{'can_explain': ['regression', 'binomial', 'mul

# Ad-hoc Run of Built-in Explainer Recipe

Run OOTB explainer recipe shipped w/ DAI server:

In [11]:
sa_explainer_id = [explainer.id for explainer in explainers if "SA explainer" == explainer.name][0]
sa_explainer_id

'h2oaicore.mli.byor.recipes.sa_explainer.SaExplainer'

In [12]:
# prepare explaination parameters
explanation_params=Client.build_common_dai_explainer_params(
    target_col=target_col,
    model_key=MODEL_KEY,
    dataset_key=DATASET_KEY,
)
explanation_params.dump()

{'common_params': {'target_col': 'default payment next month',
  'weight_col': None,
  'prediction_col': None,
  'drop_cols': None,
  'sample_num_rows': None},
 'model': {'key': '34edc42e-0152-11eb-a66d-207918bc8e4b', 'display_name': ''},
 'dataset': {'key': 'dataset_33ab9348-0152-11eb-a66d-207918bc8e4b',
  'display_name': ''},
 'validset': {'key': None, 'display_name': ''},
 'testset': {'key': None, 'display_name': ''},
 'use_raw_features': False,
 'config_overrides': None,
 'sequential_execution': True,
 'debug_model_errors': False,
 'debug_model_errors_class': ''}

In [13]:
# run explainer
explainer_id = sa_explainer_id

print(f"Running OOTB explainer: {explainer_id}")

run_job = h2oai.run_explainers(
    explainers=[Explainer(
        explainer_id=explainer_id,
        explainer_params="",
    )],
    params=explanation_params,
)
run_job.dump()

Running OOTB explainer: h2oaicore.mli.byor.recipes.sa_explainer.SaExplainer


{'explainer_job_keys': ['423581b8-017b-11eb-a66d-207918bc8e4b'],
 'mli_key': '423581b6-017b-11eb-a66d-207918bc8e4b',
 'created': 1601291446.921501,
 'duration': 0,
 'status': -1,
 'progress': 0.0}

In [14]:
# wait for explainer to finish
explainer_job_statuses = h2oai.wait_for_explainers(run_job.mli_key)
for job_status in explainer_job_statuses:
    pprint.pprint(job_status.dump())
    
mli_key = job_status.mli_key
explainer_job_key = job_status.explainer_job_key
explainer_job = job_status.explainer_job

{'explainer_job': {'child_explainers_job_keys': [],
                   'created': 1601291446.8943293,
                   'duration': 4.786961078643799,
                   'entity': {'can_explain': ['regression', 'binomial'],
                              'explanation_scopes': ['global_scope'],
                              'explanations': [{'category': 'DAI MODEL',
                                                'explanation_type': 'global-work-dir-archive',
                                                'formats': ['application/zip'],
                                                'has_local': None,
                                                'name': 'SA Data ZIP Archive',
                                                'scope': 'global'},
                                               {'category': 'DAI MODEL',
                                                'explanation_type': 'global-sensitivity-analysis',
                                                'formats': ['text/plain

## Get Recipe Result

In [15]:
# get recipe result FORMATS/TYPES (representations of recipe output)
explainer_job.entity.can_explain

['regression', 'binomial']

In [16]:
explainer_job.entity.explanation_scopes

['global_scope']

In [17]:
for explanation in explainer_job.entity.explanations: pprint.pprint(explanation.dump())

{'category': 'DAI MODEL',
 'explanation_type': 'global-work-dir-archive',
 'formats': ['application/zip'],
 'has_local': None,
 'name': 'SA Data ZIP Archive',
 'scope': 'global'}
{'category': 'DAI MODEL',
 'explanation_type': 'global-sensitivity-analysis',
 'formats': ['text/plain'],
 'has_local': None,
 'name': 'Sensitivity Analysis (SA)',
 'scope': 'global'}


In [18]:
# choose the most suitable format (if more than one) and get the result
BASE_URL = f"{address}/files/"

for explanation in explainer_job.entity.explanations:
    for e_format in explanation.formats:
        server_path: str = h2oai.get_explainer_result_url_path(
            mli_key=mli_key,
            explainer_job_key=explainer_job_key,
            explanation_type=explanation.explanation_type,
            explanation_format=e_format
        )
        print(f"Explanation {explanation.explanation_type}:\n  {e_format}:\n    {BASE_URL}{server_path}")
        
        download_dir = "/tmp"
        h2oai.download(server_path, download_dir)

Explanation global-work-dir-archive:
  application/zip:
    http://127.0.0.1:12345/files/h2oai/mli_experiment_423581b6-017b-11eb-a66d-207918bc8e4b/explainer_h2oaicore_mli_byor_recipes_sa_explainer_SaExplainer_423581b8-017b-11eb-a66d-207918bc8e4b/global_work_dir_archive/application_zip/explanation.zip
Explanation global-sensitivity-analysis:
  text/plain:
    http://127.0.0.1:12345/files/h2oai/mli_experiment_423581b6-017b-11eb-a66d-207918bc8e4b/explainer_h2oaicore_mli_byor_recipes_sa_explainer_SaExplainer_423581b8-017b-11eb-a66d-207918bc8e4b/global_sensitivity_analysis/text_plain/explanation.txt


In [19]:
!ls -l {download_dir}/explanation.zip

-rw-r--r-- 1 dvorka dvorka 1842252 Sep 28 13:10 /tmp/explanation.zip


URL from above can be used to **download** choosen recipe result representation. Explainer log can be downloaded from:

In [20]:
server_path = h2oai.get_explainer_run_log_url_path(
    mli_key=mli_key,
    explainer_job_key=explainer_job_key,
)
print(f"{BASE_URL}{server_path}")

http://127.0.0.1:12345/files/h2oai/mli_experiment_423581b6-017b-11eb-a66d-207918bc8e4b/explainer_h2oaicore_mli_byor_recipes_sa_explainer_SaExplainer_423581b8-017b-11eb-a66d-207918bc8e4b/log/explainer_run_423581b8-017b-11eb-a66d-207918bc8e4b.log


# Ad-hoc Run of Custom Explainer Recipe

Running previously uploaded custom explainer.

In [21]:
# run custom explainer - use previously uploaded recipe ID
if uploaded_explainer_id:
    explainer_id = uploaded_explainer_id
else:
    # explainer has been uploaded before
    explainers = h2oai.list_explainers(
        time_series=False, 
        dai_model_key=MODEL_KEY, 
        experiment_types=None,
        explanation_scopes=None,
        keywords=None,
    )
    for e in explainers:
        if e.name == BYOR_EXPLAINER_NAME:
            explainer_id = e.id

print(f"Running CUSTOM explainer: {explainer_id}")

run_job = h2oai.run_explainers(
    explainers=[
        Explainer(explainer_id=explainer_id, explainer_params=None)
    ],
    params=Client.build_common_dai_explainer_params(
        target_col=target_col,
        model_key=MODEL_KEY,
        dataset_key=DATASET_KEY,
    ),
)
run_job.dump()

Running CUSTOM explainer: False_morris_sensitivity_explainer_d935205d_contentexplainer.MorrisSensitivityLeExplainer


{'explainer_job_keys': ['46260112-017b-11eb-a66d-207918bc8e4b'],
 'mli_key': '46260110-017b-11eb-a66d-207918bc8e4b',
 'created': 1601291453.5125122,
 'duration': 0,
 'status': -1,
 'progress': 0.0}

In [22]:
# wait for explainer to finish
explainer_job_statuses = h2oai.wait_for_explainers(run_job.mli_key)
for job_status in explainer_job_statuses:
    pprint.pprint(job_status.dump())

{'explainer_job': {'child_explainers_job_keys': [],
                   'created': 1601291453.4908412,
                   'duration': 3.8387792110443115,
                   'entity': {'can_explain': ['regression', 'binomial'],
                              'explanation_scopes': ['global_scope'],
                              'explanations': [{'category': 'CUSTOM',
                                                'explanation_type': 'global-feature-importance',
                                                'formats': ['application/vnd.h2oai.json+datatable.jay',
                                                            'application/json'],
                                                'has_local': None,
                                                'name': 'Morris Sensitivity '
                                                        'Analysis',
                                                'scope': 'global'}],
                              'id': 'False_morris_sensitivity_explaine

In [23]:
server_path = h2oai.get_explainer_result_url_path(
    mli_key=job_status.mli_key,
    explainer_job_key=job_status.explainer_job_key,
    explanation_type='global-feature-importance',
    explanation_format='application/json'
)
print(f"{BASE_URL}{server_path}")

http://127.0.0.1:12345/files/h2oai/mli_experiment_46260110-017b-11eb-a66d-207918bc8e4b/explainer_False_morris_sensitivity_explainer_d935205d_contentexplainer_MorrisSensitivityLeExplainer_46260112-017b-11eb-a66d-207918bc8e4b/global_feature_importance/application_json/explanation.json


URL from above can be used to **download** choosen **custom** recipe result representation.

# Explain (Model) with All Compatible or Selected Explainers

In [24]:
# get IDs of previously listed recipes
explainer_ids = [explainer.id for explainer in explainers]
explainer_ids

['h2oaicore.mli.byor.recipes.sa_explainer.SaExplainer',
 'h2oaicore.mli.byor.recipes.dia_explainer.DiaExplainer',
 'h2oaicore.mli.byor.recipes.permutation_feat_imp_absolute_explainer.AbsolutePermutationFeatureImportanceExplainer',
 'h2oaicore.mli.byor.recipes.dai_pd_ice_explainer.DaiPdIceExplainer',
 'h2oaicore.mli.byor.recipes.mock.mock_markdown_explainer.MockMarkdownVegaExplainer',
 'h2oaicore.mli.byor.recipes.mock.mock_pd_explainer.MockPartialDependenceExplainer',
 'h2oaicore.mli.byor.recipes.mock.mock_dt_explainer.MockDecisionTreeExplainer',
 'h2oaicore.mli.byor.recipes.mock.mock_markdown_pd_explainer.MockMarkdownPdExplainer',
 'h2oaicore.mli.byor.recipes.mock.mock_featimp_explainer.MockFeatureImportanceExplainer',
 'h2oaicore.mli.byor.recipes.mock.mock_scatter_plot_explainer.MockScatterPlotExplainer',
 'h2oaicore.mli.byor.recipes.surrogates_n_shapleys_explainer.MainSurrogatesAndShapleysExplainer',
 'h2oaicore.mli.byor.recipes.permutation_feat_imp_relative_explainer.RelativePermuta

In [25]:
# run explainers: list of IDs OR empty list
#   - empty explainer IDs list means "run all model COMPATIBLE explainers with default parameters")

print(f"All explainers:\n{explainer_ids}")
run_job: ExplainersRunJob = h2oai.run_explainers(
    explainers=[],
    params=Client.build_common_dai_explainer_params(
        target_col=target_col,
        model_key=MODEL_KEY,
        dataset_key=DATASET_KEY,
    ),
)
run_job.dump()

All explainers:
['h2oaicore.mli.byor.recipes.sa_explainer.SaExplainer', 'h2oaicore.mli.byor.recipes.dia_explainer.DiaExplainer', 'h2oaicore.mli.byor.recipes.permutation_feat_imp_absolute_explainer.AbsolutePermutationFeatureImportanceExplainer', 'h2oaicore.mli.byor.recipes.dai_pd_ice_explainer.DaiPdIceExplainer', 'h2oaicore.mli.byor.recipes.mock.mock_markdown_explainer.MockMarkdownVegaExplainer', 'h2oaicore.mli.byor.recipes.mock.mock_pd_explainer.MockPartialDependenceExplainer', 'h2oaicore.mli.byor.recipes.mock.mock_dt_explainer.MockDecisionTreeExplainer', 'h2oaicore.mli.byor.recipes.mock.mock_markdown_pd_explainer.MockMarkdownPdExplainer', 'h2oaicore.mli.byor.recipes.mock.mock_featimp_explainer.MockFeatureImportanceExplainer', 'h2oaicore.mli.byor.recipes.mock.mock_scatter_plot_explainer.MockScatterPlotExplainer', 'h2oaicore.mli.byor.recipes.surrogates_n_shapleys_explainer.MainSurrogatesAndShapleysExplainer', 'h2oaicore.mli.byor.recipes.permutation_feat_imp_relative_explainer.RelativePe

{'explainer_job_keys': ['49ec5ea4-017b-11eb-a66d-207918bc8e4b',
  '49ec5ea6-017b-11eb-a66d-207918bc8e4b',
  '49ec5ea8-017b-11eb-a66d-207918bc8e4b',
  '49ec5eaa-017b-11eb-a66d-207918bc8e4b',
  '49ec5eac-017b-11eb-a66d-207918bc8e4b',
  '49ec5eae-017b-11eb-a66d-207918bc8e4b',
  '49ec5eb0-017b-11eb-a66d-207918bc8e4b',
  '49ec5eb2-017b-11eb-a66d-207918bc8e4b',
  '49ec5eb4-017b-11eb-a66d-207918bc8e4b',
  '49ec5eb6-017b-11eb-a66d-207918bc8e4b',
  '49ec5eb8-017b-11eb-a66d-207918bc8e4b',
  '49ec5eba-017b-11eb-a66d-207918bc8e4b'],
 'mli_key': '49ec5ea2-017b-11eb-a66d-207918bc8e4b',
 'created': 1601291460.452971,
 'duration': 0,
 'status': -1,
 'progress': 0.0}

In [26]:
# check interpretation job status (legacy RPC API)
i_job: InterpretationJob = h2oai.get_interpretation_job(run_job.mli_key)
# note per-explainer (subtask) ID and display name
i_job.dump()

{'progress': 0.1,
 'h2oprogress': {'progress': 0.1, 'msg': '', 'done': False},
 'mliprogress': {'progress': 0.1, 'msg': '', 'done': False},
 'status': -1,
 'error': '',
 'message': 'Explainers running',
 'entity': {'key': '49ec5ea2-017b-11eb-a66d-207918bc8e4b',
  'description': '49ec5ea2-017b-11eb-a66d-207918bc8e4b',
  'tmp_dir': './tmp',
  'scoring_package_path': '',
  'binned_list': [],
  'labels': [],
  'parameters': {'dai_model': {'key': '34edc42e-0152-11eb-a66d-207918bc8e4b',
    'display_name': ''},
   'dataset': {'key': 'dataset_33ab9348-0152-11eb-a66d-207918bc8e4b',
    'display_name': ''},
   'target_col': 'default payment next month',
   'prediction_col': None,
   'use_raw_features': False,
   'nfolds': -1,
   'klime_cluster_col': '',
   'weight_col': None,
   'drop_cols': None,
   'sample': False,
   'sample_num_rows': -1,
   'qbin_cols': [],
   'qbin_count': -1,
   'lime_method': '',
   'dt_tree_depth': -1,
   'config_overrides': '',
   'vars_to_pdp': -1,
   'dia_cols': [],

In [27]:
# check particular sub-job status (existing RPC API reused)
h2oai.get_explainer_job_status(run_job.mli_key, run_job.explainer_job_keys[0]).dump()

{'mli_key': '49ec5ea2-017b-11eb-a66d-207918bc8e4b',
 'explainer_job_key': '49ec5ea4-017b-11eb-a66d-207918bc8e4b',
 'explainer_job': {'progress': 0,
  'status': -1,
  'error': '',
  'message': 'Entering sequential execution gate...',
  'entity': {'id': 'h2oaicore.mli.byor.recipes.sa_explainer.SaExplainer',
   'name': 'SA explainer',
   'model_types': ['iid'],
   'can_explain': ['regression', 'binomial'],
   'explanation_scopes': ['global_scope'],
   'explanations': [],
   'keywords': ['run-by-default']},
  'created': 1601291459.8144472,
  'duration': 0.6814770698547363,
  'child_explainers_job_keys': []}}

In [28]:
# check sub-jobs statuses (existing RPC API reused)
job_statuses = h2oai.get_explainer_job_statuses(run_job.mli_key, run_job.explainer_job_keys)
for js in job_statuses:
    pprint.pprint(js.dump())

{'explainer_job': {'child_explainers_job_keys': [],
                   'created': 1601291459.8144472,
                   'duration': 0.6814770698547363,
                   'entity': {'can_explain': ['regression', 'binomial'],
                              'explanation_scopes': ['global_scope'],
                              'explanations': [],
                              'id': 'h2oaicore.mli.byor.recipes.sa_explainer.SaExplainer',
                              'keywords': ['run-by-default'],
                              'model_types': ['iid'],
                              'name': 'SA explainer'},
                   'error': '',
                   'message': 'Entering sequential execution gate...',
                   'progress': 0,
                   'status': -1},
 'explainer_job_key': '49ec5ea4-017b-11eb-a66d-207918bc8e4b',
 'mli_key': '49ec5ea2-017b-11eb-a66d-207918bc8e4b'}
{'explainer_job': {'child_explainers_job_keys': [],
                   'created': 1601291459.8862343,
     

                   'created': 1601291460.125687,
                   'duration': 0.4347710609436035,
                   'entity': {'can_explain': ['regression', 'binomial'],
                              'explanation_scopes': ['global_scope',
                                                     'local_scope'],
                              'explanations': [],
                              'id': 'h2oaicore.mli.byor.recipes.surrogates.klime_explainer.KLimeExplainer',
                              'keywords': ['run-by-default', 'proxy-explainer'],
                              'model_types': ['iid'],
                              'name': 'k-LIME explainer'},
                   'error': '',
                   'message': 'Entering sequential execution gate...',
                   'progress': 0,
                   'status': -1},
 'explainer_job_key': '49ec5eb2-017b-11eb-a66d-207918bc8e4b',
 'mli_key': '49ec5ea2-017b-11eb-a66d-207918bc8e4b'}
{'explainer_job': {'child_explainers_job_keys': [],


In [29]:
# wait for ALL explainers to finish
explainer_statuses=h2oai.wait_for_explainers(run_job.mli_key)
explainer_statuses

In [30]:
# download explanation type in desired format
DOWNLOAD_DIR = f"/tmp/interpretation_run_{randint(0,1_000_000)}"

explainer_job_key = explainer_statuses[0].explainer_job_key
explanations = h2oai.list_explainer_results(explainer_job_key=explainer_job_key).explanations
# explanations
for explanation in explanations:
    # explantion's formats
    for explanation_format in explanation.formats:
        # format's URL
        result_path: str = h2oai.get_explainer_result_url_path(
            mli_key=run_job.mli_key,
            explainer_job_key=explainer_job_key,
            explanation_type=explanation.explanation_type,
            explanation_format=explanation_format,
        )
            
        # where to download
        EXPLANATION_DIR = f"{DOWNLOAD_DIR}/explanation_{randint(0,1_000_000)}"
        os.makedirs(EXPLANATION_DIR, exist_ok=True)
        
        # download
        h2oai.download(result_path, EXPLANATION_DIR)

        print(
            f"Explanation {explanation.explanation_type}:\n"
            f"  {explanation_format}:\n"
            f"    {BASE_URL}{result_path}"
        )
        
print(f"\nDownloaded explanations in {DOWNLOAD_DIR}:")
!ls -l {DOWNLOAD_DIR}

Explanation global-work-dir-archive:
  application/zip:
    http://127.0.0.1:12345/files/h2oai/mli_experiment_49ec5ea2-017b-11eb-a66d-207918bc8e4b/explainer_h2oaicore_mli_byor_recipes_sa_explainer_SaExplainer_49ec5ea4-017b-11eb-a66d-207918bc8e4b/global_work_dir_archive/application_zip/explanation.zip
Explanation global-sensitivity-analysis:
  text/plain:
    http://127.0.0.1:12345/files/h2oai/mli_experiment_49ec5ea2-017b-11eb-a66d-207918bc8e4b/explainer_h2oaicore_mli_byor_recipes_sa_explainer_SaExplainer_49ec5ea4-017b-11eb-a66d-207918bc8e4b/global_sensitivity_analysis/text_plain/explanation.txt

Downloaded explanations in /tmp/interpretation_run_388854:
total 8
drwxr-xr-x 2 dvorka dvorka 4096 Sep 28 13:16 explanation_753563
drwxr-xr-x 2 dvorka dvorka 4096 Sep 28 13:16 explanation_809124
